In [2]:
import json

with open('dataset.json', 'r') as f:
    dataset = json.load(f)

print(dataset)


[{'user_input': "What's the weather like today?", 'intent': 'get_weather', 'entities': {'date': 'today'}}, {'user_input': 'Book a flight to New York tomorrow.', 'intent': 'book_flight', 'entities': {'destination': 'New York', 'date': 'tomorrow'}}, {'user_input': 'Set a reminder for 9 AM.', 'intent': 'set_reminder', 'entities': {'time': '9 AM'}}, {'user_input': 'Order me a pizza.', 'intent': 'order_food', 'entities': {'food_item': 'pizza'}}, {'user_input': 'Find me a nearby coffee shop.', 'intent': 'find_place', 'entities': {'place_type': 'coffee shop', 'location': 'nearby'}}, {'user_input': 'Play some jazz music.', 'intent': 'play_music', 'entities': {'genre': 'jazz'}}, {'user_input': 'Show me the news.', 'intent': 'get_news', 'entities': {}}, {'user_input': "What's the capital of France?", 'intent': 'get_fact', 'entities': {'country': 'France'}}, {'user_input': "What's 25 times 6?", 'intent': 'math_operation', 'entities': {'operation': 'multiplication', 'numbers': [25, 6]}}, {'user_in

In [3]:
from transformers import pipeline

intent_model = pipeline("zero-shot-classification")

entity_model = pipeline("ner", aggregation_strategy="simple")


No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

In [4]:
def extract_intent(user_input):
    labels = [entry['intent'] for entry in dataset]
    result = intent_model(user_input, candidate_labels=labels)
    return result

def extract_entities(user_input):
    entities = entity_model(user_input)
    return entities


In [5]:
feedback_store = []

def collect_feedback(user_input, rating):
    feedback_store.append({"input": user_input, "rating": rating})

def adjust_responses():
    for feedback in feedback_store:
        if feedback["rating"] < 3:
            print(f"Improve response for: {feedback['input']}")


In [6]:
def chatbot_response(user_input):
    # Extract intent and entities
    intent_result = extract_intent(user_input)
    entities_result = extract_entities(user_input)

    # Get the highest scored intent
    best_intent = intent_result['labels'][0]
    best_score = intent_result['scores'][0]

    # Extract entities
    entities = [{ "word": entity['word'], "score": entity['score'] } for entity in entities_result]

    # Generate a response based on intent (this is a placeholder)
    response = f"Intent: {best_intent}\nEntities: {entities}"

    # Return response to the user
    return response

# Example usage
user_input = "Please Cancel my 8AM appointment In Japan"
response = chatbot_response(user_input)
print(response)

# Collect feedback
rating = int(input("Rate the response from 1 to 5: "))
collect_feedback(user_input, rating)
adjust_responses()


Intent: cancel_alarm
Entities: [{'word': 'Japan', 'score': 0.9990658}]
Rate the response from 1 to 5: 4
